In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, train_test_split
import pandas as pd

import sys
import os
module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(1, module_path + '/utils/')
import valid_recs as vrecs
import data
import labels
import features
import classifiers as cls
import variables as v

In [ ]:
valid_recs = vrecs.get_valid_recs(data_type='ica', output_type = 'mne')

x_dict = data.extract_eeg_data(valid_recs, data_type='ica', output_type='mne')


In [ ]:
y_dict = labels.get_stai_labels(valid_recs) 
#y_dict =labels.get_pss_labels(valid_recs)

In [ ]:
print(len(x_dict))
print(len(y_dict))

In [ ]:
targ_val = 1
rem=[]
for i in y_dict.keys():
    if y_dict[i] is targ_val:
        rem.append(i)
# printing result
print("\nThe extracted keys : \n" + str(rem))

[y_dict.pop(key) for key in rem]
[x_dict.pop(key) for key in rem]
print(f"\nDictionary after removal of keys from y_dict: \n {y_dict.keys()}")
print(f"\nDictionary after removal of keys from x_dict: \n {x_dict.keys()}")

In [ ]:
x_epochs, y_epochs = data.segment_data(x_dict, y_dict, epoch_duration=3)


In [ ]:
#print(x_epochs)

#x_epochs_df = pd.DataFrame.from_dict(x_epochs)


In [ ]:
splits = data.kfold_split(x_epochs, y_epochs, n_splits=2, shuffle=True, random_state=42)
train_epochs, test_epochs, train_labels, test_labels = splits
y_train = [np.array([v for _, v in train_labels[i].items()]) for i in range(len(train_labels))]
y_test = [np.array([v for _, v in test_labels[i].items()]) for i in range(len(test_labels))]


#x, x_test, y, test_labels = train_test_split(x_epochs, y_epochs, test_size=0.2, random_state=1)
#x_train, x_val, train_labels, val_labels = train_test_split(x, y, test_size=0.25, random_state=1)


# freq_bands = np.array([1, 4, 8, 12, 30, 50])
# features = freq_band_features(dataset, freq_bands)
# features = hjorth_features(dataset)
# features = entropy_features(dataset)
#  features = fractal_features(dataset)

In [ ]:
x_train = features.time_series_features(train_epochs) 
x_test = features.time_series_features(test_epochs)

In [ ]:
print(x_train[0].shape)

print(len(train_epochs[0]))
print(train_epochs[0]['P001_S001_001_epoch0'].shape)
print(train_epochs[0]['P001_S001_001_epoch0'][7].shape[-1])

In [ ]:
"""
Compute the scattering transform of a speech recording
======================================================
This script loads a speech signal consisting of an excerpt from a recording of
*Sense and Sensibility*. We then compute its scattering transform, and display
the zeroth-, first-, and second-order scattering coefficients.
"""

###############################################################################
# Preliminaries
# -------------
#
###############################################################################
# To manipulate the audio signal, we first import NumPy. We also import
# `librosa`, which allows us to automatically download the example signal.

import numpy as np
import librosa

###############################################################################
# We import `matplotlib` to plot the calculated scattering coefficients.

import matplotlib.pyplot as plt

###############################################################################
# Finally, we import the `Scattering1D` class from the `scattering` package.
# The `Scattering1D` class is what lets us calculate the scattering transform

from kymatio.numpy import Scattering1D

###############################################################################
# Scattering setup
# ----------------
# First, we download the signal and extract the second second of it (the first
# second is mostly silence).

#x, sr = librosa.load(librosa.example("libri3"))
#x = x[sr:2 * sr]

###############################################################################
# Once the recording is in memory, we normalize it.
x = train_epochs[0]['P001_S001_001_epoch0'][7]
x = x / np.max(np.abs(x))

###############################################################################
# We are now ready to set up the parameters for the scattering transform.
# First, the number of samples, `T`, is given by the size of our input `x`.
# The averaging scale is specified as a power of two, `2**J`. Here, we set
# `J = 6` to get an averaging, or maximum, scattering scale of `2**6 = 64`
# samples. Finally, we set the number of wavelets per octave, `Q`, to `16`.
# This lets us resolve frequencies at a resolution of `1/16` octaves.

T = x.shape[-1]
print(T)
J = 6
Q = 16

###############################################################################
# Finally, we are able to create the object which computes our scattering
# transform, `scattering`.

scattering = Scattering1D(J, T, Q)

###############################################################################
# Compute and display the scattering coefficients
# -----------------------------------------------
# Computing the scattering transform of a signal is achieved using the
# `__call__` method of the `Scattering1D` class. The output is an array of
# shape `(C, T)`. Here, `C` is the number of scattering coefficient outputs,
# and `T` is the number of samples along the time axis. This is typically much
# smaller than the number of input samples since the scattering transform
# performs an average in time and subsamples the result to save memory.

Sx = scattering(x)
print('Sx shape: ', Sx.shape)
###############################################################################
# To display the scattering coefficients, we must first identify which belong
# to each order (zeroth, first, or second). We do this by extracting the `meta`
# information from the scattering object and constructing masks for each order.

meta = scattering.meta()
order0 = np.where(meta['order'] == 0)
order1 = np.where(meta['order'] == 1)
order2 = np.where(meta['order'] == 2)

###############################################################################
# First, we plot the original signal `x`.

plt.figure(figsize=(8, 2))
plt.plot(x)
plt.title('Original signal')
plt.show()

###############################################################################
# We now plot the zeroth-order scattering coefficient, which is simply an
# average of the original signal at the scale `2**J`.

plt.figure(figsize=(8, 8))
plt.plot(Sx[order0][0])
plt.title('Zeroth-order scattering')
plt.show()
print('len order 0: ', Sx[order0].shape)

###############################################################################
# We then plot the first-order coefficients, which are arranged along time
# and log-frequency.

plt.imshow(Sx[order1], aspect='auto')
plt.title('First-order scattering')
plt.show()
print('len order 1: ', Sx[order1].shape)

###############################################################################
# Finally, we plot the second-order scattering coefficients. These are also
# organized aling time, but has two log-frequency indices: one first-order
# frequency and one second-order frequency. Here, both indices are mixed along
# the vertical axis.

plt.imshow(Sx[order2], aspect='auto')
plt.title('Second-order scattering')
plt.show()
print('len order 2: ', Sx[order2].shape)
print(Sx.shape)

Sx= np.mean(Sx, axis=-1)
Sx = np.ndarray.flatten(Sx)
print(np.transpose(Sx.shape))
print(np.transpose(Sx))

In [ ]:
x_train = features.kymatio_wave_scattering(train_epochs) 

In [ ]:
print(x_train)

In [ ]:
x_test = features.kymatio_wave_scattering(test_epochs)

In [ ]:
print(x_test)

### KNN

In [ ]:
print('ica data')
cls.knn_classification(x_train, x_test, y_train, y_test)

### SVM

In [ ]:
print('ica data')
print(cls.svm_classification(x_train, x_test, y_train, y_test))
